In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()

import imaplib
import email
import pandas as pd
from datetime import datetime
import csv
from pandas import DataFrame

from email.header import decode_header
from bs4 import BeautifulSoup
import re
from apps.email_manager.models import EmailData, EmailAttachment
from django.core.files.base import ContentFile
import pytz

# what is the longest length of convo
# how to extract one conversations from the data

In [68]:
class EmailNode:
    def __init__(self, previous_node, email_data, next_node):
        self.previous_node: EmailNode = previous_node
        self.email_data: EmailData = email_data
        self.next_node: EmailNode = next_node

class EmailConvo:
    def __init__(self, first_email):
        email_node = EmailNode(
            previous_node=None,
            email_data=first_email,
            next_node=None
        )
        self.head_email_node = email_node
        self.tail_email_node = email_node
    
    def append_email_node_to_convo(self, new_email):
        next_node = EmailNode(
            previous_node=self.tail_email_node,
            email_data=new_email,
            next_node=None
        )
        self.tail_email_node.next_node = next_node
        self.tail_email_node = next_node

    def prepend_email_node_to_convo(self, new_email):
        first_node = EmailNode(
            previous_node=None,
            email_data=new_email,
            next_node=self.head_email_node
        )
        self.head_email_node.previous_node = first_node
        self.head_email_node = first_node

    def remove_last_email(self):
        new_last_email = self.tail_email_node.previous_node
        if new_last_email is not None:
            new_last_email.next_node = None
            self.tail_email_node = new_last_email

    def remove_first_email(self):
        new_first_email = self.head_email_node.next_node
        if new_first_email is not None:
            new_first_email.previous_node = None
            self.head_email_node = new_first_email
    
    def remove_email_from_convo(self, index = None, e_id = None):
        if (index is not None and id is not None) or (index is None and id is None):
            return -1
        iter = 0
        def _remove_email_from_convo(current_node: EmailNode):
            nonlocal iter
            if (e_id is not None and current_node.email_data.id == e_id) or (index is not None and iter==index):
                previous_node: EmailNode = current_node.previous_node
                next_node: EmailNode = current_node.next_node
                if previous_node is not None and next_node is not None:
                    previous_node.next_node = next_node
                    next_node.previous_node = previous_node
                elif previous_node is not None and next_node is None:
                    previous_node.next_node = None
                elif previous_node is None and next_node is not None:
                    next_node.previous_node = None
                    self.head_email_node = next_node
                elif previous_node is None and next_node is None:
                    return 0
                return 1
            else:
                next_node = current_node.next_node
                if next_node is not None:
                    iter = iter + 1
                    return _remove_email_from_convo(next_node)
                else:
                    return -1
        return _remove_email_from_convo(self.head_email_node)
    
    def insert_email_into_convo(self, new_email: EmailData, index = None, e_id = None):
        if (index is not None and id is not None) or (index is None and id is None):
            return -1
        iter = 0
        def _insert_email_into_convo(current_node: EmailNode):
            nonlocal iter
            if (e_id is not None and current_node.email_data.id == e_id) or (index is not None and iter==index):
                new_email_node = EmailNode(
                    previous_node=None,
                    email_data=new_email,
                    next_node=None
                )
                previous_node: EmailNode = current_node.previous_node
                next_node: EmailNode = current_node.next_node
                if previous_node is not None:
                    previous_node.next_node = new_email_node
                    new_email_node.previous_node = previous_node

                    current_node.previous_node = new_email_node
                    new_email_node.next_node = current_node

                else:
                    current_node.previous_node = new_email_node
                    new_email_node.next_node = current_node
                    self.head_email_node = new_email_node
                return 1
            else:
                next_node = current_node.next_node
                if next_node is not None:
                    iter = iter + 1
                    return _insert_email_into_convo(next_node)
                else:
                    return -1
        return _insert_email_into_convo(self.head_email_node)
    
    def retrieve_email_from_convo(self, index = None, e_id = None, text = None):
        if (index is None and id is None and text is None):
            return -1
        iter = 0
        def _retrieve_email_from_convo(current_node: EmailNode):
            nonlocal iter
            if (
                (e_id is not None and current_node.email_data.id == e_id) 
                or (index is not None and iter==index) 
                or (current_node.email_data.body is not None and text in current_node.email_data.body)
            ):
                return current_node.email_data
            else:
                next_node = current_node.next_node
                if next_node is not None:
                    iter = iter + 1
                    return _retrieve_email_from_convo(next_node)
                else:
                    return None
        return _retrieve_email_from_convo(self.head_email_node)
    
    def get_full_convo(self):
        first_email = self.head_email_node
        full_convo = []
        def _get_full_convo(current_email: EmailNode):
            email_data = current_email.email_data
            full_convo.append({
                "from": email_data.from_email,
                "to": email_data.to_emails,
                "date": email_data.date.strftime('%B %d, %Y, %-I:%M %p'),
                "body": email_data.body
            })
            next_email = current_email.next_node
            if next_email is not None:
                return _get_full_convo(next_email)
            else:
                return None
        _get_full_convo(first_email)
        return full_convo

            

        


In [69]:
# if it is of the form re, but before the re, there is one without the re, then split
class ConversationReader:
    def __init__(self):
        self.conversations = {}
        self.latest_subjects = {}
        self.re_list = [
            "RE: ",
            "Re: ",
            "re: "
        ]
        self.fwd_list = [
            "FW: ",
            "fw: ",
            "Fw: ",
            "FWD: ",
            "Fwd: ",
            "fwd: "
        ]

    def extract_conversation(self):
        emails = EmailData.objects.order_by('date').all()[0:1500]
        for an_email in emails:
            if an_email.subject and an_email.date:
                is_reply = False
                is_fwdd = False
                original_subject = an_email.subject.strip()
                
                if any(substring in original_subject for substring in self.re_list):
                    is_reply = True
                if any(substring in original_subject for substring in self.fwd_list):
                    is_fwdd = True
                rmvd_subject = original_subject
                for re_substring in self.re_list:
                    rmvd_subject = rmvd_subject.replace(re_substring, "")
                for fwd_substring in self.fwd_list:
                    rmvd_subject = rmvd_subject.replace(fwd_substring, "")
                rmvd_subject = rmvd_subject.replace("\n", "").replace("\r", "")
                # if 'RENEWAL | M/S HYGIENE CONTAINERS (PRIVATE) LIMITED | FIRE POLICIES' in rmvd_subject:
                #     print(original_subject)
                if rmvd_subject in self.latest_subjects:
                    if not is_reply and not is_fwdd:
                        latest_date = an_email.date
                        self.latest_subjects[rmvd_subject] = latest_date
                    else:
                        latest_date = self.latest_subjects[rmvd_subject]
                else:
                    latest_date = an_email.date
                    self.latest_subjects[rmvd_subject] = latest_date
                formatted_date = latest_date.strftime('%B %d, %Y, %-I:%M %p')
                key_str = f"{formatted_date}@{rmvd_subject}"
                if key_str in self.conversations:
                    convo_emails: EmailConvo = self.conversations[key_str]
                    convo_emails.append_email_node_to_convo(an_email)
                else:
                    self.conversations[key_str] = EmailConvo(first_email=an_email)
                


In [70]:
convo_reader = ConversationReader()
convo_reader.extract_conversation()

In [71]:
# keys_list = convo_reader.latest_subjects.keys()
# for a_key in keys_list:
#     if "RENEWAL | M/S HYGIENE CONTAINERS" in a_key:
#         print(a_key)
#         print("-")
#         "RE: "
# print(convo_reader.latest_subjects["RENEWAL | M/S HYGIENE CONTAINERS (PRIVATE) LIMITED | FIRE POLICIES"])
subj = 'RENEWAL | M/S HYGIENE CONTAINERS (PRIVATE) LIMITED | FIRE POLICIES'
my_date = formatted_date = convo_reader.latest_subjects[subj].strftime('%B %d, %Y, %-I:%M %p')
key_str = f"May 23, 2023, 12:02 PM@{subj}"
print(key_str)
a_convo: EmailConvo = convo_reader.conversations[key_str]

May 23, 2023, 12:02 PM@RENEWAL | M/S HYGIENE CONTAINERS (PRIVATE) LIMITED | FIRE POLICIES


KeyError: 'May 23, 2023, 12:02 PM@RENEWAL | M/S HYGIENE CONTAINERS (PRIVATE) LIMITED | FIRE POLICIES'

In [72]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'INVOICE FOR INSURANCE COVER' in a_convo_str:
        full_convo = a_convo_data.get_full_convo()
        for a_msg in full_convo:
            print(a_msg)
        print("")


{'from': 'Sales Adeel <Sales.Adeel@adamjeeinsurance.com>', 'to': ['Muhammad Feroze Vakil <Feroze.Vakil@adamjeeinsurance.com>'], 'date': 'May 08, 2023, 9:18 AM', 'body': 'Dear Feroze bhai,\n\xa0\nKindly issue cover notes.\n\xa0\n'}

{'from': 'Abdul Rashid <Abdul.Rashid@adamjeeinsurance.com>', 'to': ['Shahzad Khan <shahzad@almuqeettextiles.com>, Mir Babar Ali\r\n\t<Mir.babarali@adamjeeinsurance.com>'], 'date': 'May 08, 2023, 9:22 AM', 'body': 'Please attached invoice ?\n\xa0\n'}

{'from': 'Sales Adeel <Sales.Adeel@adamjeeinsurance.com>', 'to': ['Abdul Rashid <Abdul.Rashid@adamjeeinsurance.com>, Shahzad Khan\r\n\t<shahzad@almuqeettextiles.com>, Mir Babar Ali\r\n\t<Mir.babarali@adamjeeinsurance.com>'], 'date': 'May 08, 2023, 9:24 AM', 'body': 'Dear Rashid bhai,\n\xa0\nFYI.\n\xa0\n'}

{'from': '"Shahzad Khan" <shahzad@almuqeettextiles.com>', 'to': ['"\'Mir Babar Ali\'" <Mir.babarali@adamjeeinsurance.com>'], 'date': 'May 08, 2023, 11:01 AM', 'body': 'Dear Mir Babar,\n\xa0\nPlease make a cove